# This is the beginning of the DataHacks ScienceGang Project

## Part1: General Setup and Imports

In [27]:
# general imports
import os
import numpy as np
import pandas as pd
import seaborn as sns
import folium
from folium import plugins

# These lines set up graphing capabilities.
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
pd.set_option('display.max_rows', 15)

%matplotlib inline

ModuleNotFoundError: No module named 'folium'

In [2]:
# read .csv file into a DataFrame
loc = 'U.S._Chronic_Disease_Indicators__CDI_.csv'
Chronic_Diseases = pd.read_csv(loc)
Chronic_Diseases

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2015,2015,US,United States,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,59,ALC,ALC8_0_2,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
1,2012,2012,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,11,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
2,2012,2012,GU,Guam,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,66,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
3,2012,2012,PR,Puerto Rico,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,72,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
4,2012,2012,US,United States,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,NaN,$,US Dollars,...,59,ALC,ALC8_0_1,USD,OVERALL,OVR,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519713,2014,2014,DE,Delaware,BRFSS,Arthritis,Physical inactivity among adults aged >= 18 ye...,NaN,%,Age-adjusted Prevalence,...,10,ART,ART3_0,AGEADJPREV,RACE,OTH,NaN,NaN,NaN,NaN
519714,2015,2015,OR,Oregon,BRFSS,"Nutrition, Physical Activity, and Weight Status",Meeting aerobic physical activity guidelines f...,NaN,%,Crude Prevalence,...,41,NPAW,NPAW11_3,CRDPREV,GENDER,GENF,NaN,NaN,NaN,NaN
519715,2014,2014,AR,Arkansas,State Inpatient Data,Chronic Obstructive Pulmonary Disease,Hospitalization for chronic obstructive pulmon...,NaN,"cases per 10,000",Crude Rate,...,5,COPD,COPD5_1,CRDRATE,GENDER,GENF,NaN,NaN,NaN,NaN
519716,2016,2016,CT,Connecticut,BRFSS,Cardiovascular Disease,Influenza vaccination among noninstitutionaliz...,NaN,%,Age-adjusted Prevalence,...,9,CVD,CVD9_2,AGEADJPREV,RACE,HIS,NaN,NaN,NaN,NaN


In [3]:
# browse data structure and contents
print(Chronic_Diseases.shape)
print(Chronic_Diseases.describe())

(519718, 34)
           YearStart        YearEnd  DataValueAlt  LowConfidenceLimit  \
count  519718.000000  519718.000000  3.503350e+05       311062.000000   
mean     2013.141885    2013.162754  8.917742e+02           46.759401   
std         1.777622       1.762672  1.833073e+04           77.492628   
min      2001.000000    2001.000000  0.000000e+00            0.200000   
25%      2012.000000    2012.000000  1.845500e+01           12.700000   
50%      2013.000000    2013.000000  4.100000e+01           30.200000   
75%      2015.000000    2015.000000  7.030000e+01           55.400000   
max      2016.000000    2016.000000  2.600878e+06         1330.660000   

       HighConfidenceLimit     LocationID  
count        311062.000000  519718.000000  
mean             58.991287      30.993144  
std              88.668862      17.723341  
min               0.420000       1.000000  
25%              18.900000      17.000000  
50%              43.800000      30.000000  
75%              70.4

In [4]:
Chronic_Diseases.isna().sum()

YearStart                         0
YearEnd                           0
LocationAbbr                      0
LocationDesc                      0
DataSource                        0
                              ...  
StratificationID1                 0
StratificationCategoryID2    440529
StratificationID2            440529
StratificationCategoryID3    440530
StratificationID3            440530
Length: 34, dtype: int64

## Explore Empty Columns

In [5]:
responses = Chronic_Diseases['Response'].value_counts()
print(Chronic_Diseases.get("Response").isna().sum())
responses

440530


     79188
Name: Response, dtype: int64

In [6]:
responses = Chronic_Diseases['StratificationCategoryID2'].value_counts()
print(Chronic_Diseases.get("StratificationCategoryID2").isna().sum())
responses

440529


     79189
Name: StratificationCategoryID2, dtype: int64

In [7]:
responses = Chronic_Diseases['StratificationID2'].value_counts()
print(Chronic_Diseases.get("StratificationID2").isna().sum())
responses

440529


     79189
Name: StratificationID2, dtype: int64

In [8]:
responses = Chronic_Diseases['StratificationCategoryID3'].value_counts()
print(Chronic_Diseases.get("StratificationCategoryID3").isna().sum())
responses

440530


     79188
Name: StratificationCategoryID3, dtype: int64

In [9]:
responses = Chronic_Diseases['StratificationID3'].value_counts()
print(Chronic_Diseases.get("StratificationID3").isna().sum())
responses

440530


     79188
Name: StratificationID3, dtype: int64

In [10]:
responses = Chronic_Diseases['StratificationCategory3'].value_counts()
print(Chronic_Diseases.get("StratificationCategory3").isna().sum())
responses

440530


     79188
Name: StratificationCategory3, dtype: int64

In [11]:
responses = Chronic_Diseases['Stratification3'].value_counts()
print(Chronic_Diseases.get("Stratification3").isna().sum())
responses

440530


     79188
Name: Stratification3, dtype: int64

In [12]:
responses = Chronic_Diseases['StratificationCategory2'].value_counts()
print(Chronic_Diseases.get("StratificationCategory2").isna().sum())
responses

440530


     79188
Name: StratificationCategory2, dtype: int64

In [13]:
responses = Chronic_Diseases['Stratification2'].value_counts()
print(Chronic_Diseases.get("Stratification2").isna().sum())
responses

440530


     79188
Name: Stratification2, dtype: int64

In [14]:
responses = Chronic_Diseases['ResponseID'].value_counts()
print(Chronic_Diseases.get("ResponseID").isna().sum())
responses

440542


     79176
Name: ResponseID, dtype: int64

In [15]:
responses = Chronic_Diseases['DataValueFootnoteSymbol'].value_counts()
print(Chronic_Diseases.get('DataValueFootnoteSymbol').isna().sum())
responses

292400


****     98370
         56098
-        39252
~        30532
*         2062
         ...  
*, #         6
§§           1
§,‡          1
§,‡,^        1
^^^          1
Name: DataValueFootnoteSymbol, Length: 19, dtype: int64

Since there is no meaningful data in these columns, they will be dropped.

In [16]:
Chronic_Diseases = Chronic_Diseases.drop(columns = ['Response', 'StratificationCategoryID2', 'StratificationID2',
                        'StratificationCategoryID3', 'StratificationID3', 'StratificationCategory3',
                       'Stratification3', 'StratificationCategory2', 'Stratification2',
                       'ResponseID', 'DataValueFootnoteSymbol'])
Chronic_Diseases

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,...,HighConfidenceLimit,StratificationCategory1,Stratification1,GeoLocation,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1
0,2015,2015,US,United States,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,$,US Dollars,NaN,...,NaN,Overall,Overall,NaN,59,ALC,ALC8_0_2,USD,OVERALL,OVR
1,2012,2012,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,$,US Dollars,0.09,...,NaN,Overall,Overall,"(38.907192, -77.036871)",11,ALC,ALC8_0_1,USD,OVERALL,OVR
2,2012,2012,GU,Guam,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,$,US Dollars,NaN,...,NaN,Overall,Overall,"(13.444304, 144.793731)",66,ALC,ALC8_0_1,USD,OVERALL,OVR
3,2012,2012,PR,Puerto Rico,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,$,US Dollars,NaN,...,NaN,Overall,Overall,"(18.220833, -66.590149)",72,ALC,ALC8_0_1,USD,OVERALL,OVR
4,2012,2012,US,United States,APIS,Alcohol,Amount of alcohol excise tax by beverage type ...,$,US Dollars,NaN,...,NaN,Overall,Overall,NaN,59,ALC,ALC8_0_1,USD,OVERALL,OVR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519713,2014,2014,DE,Delaware,BRFSS,Arthritis,Physical inactivity among adults aged >= 18 ye...,%,Age-adjusted Prevalence,NaN,...,NaN,Race/Ethnicity,"Other, non-Hispanic","(39.008830667, -75.577741168)",10,ART,ART3_0,AGEADJPREV,RACE,OTH
519714,2015,2015,OR,Oregon,BRFSS,"Nutrition, Physical Activity, and Weight Status",Meeting aerobic physical activity guidelines f...,%,Crude Prevalence,39.5,...,42.00,Gender,Female,"(44.567449424, -120.155031326)",41,NPAW,NPAW11_3,CRDPREV,GENDER,GENF
519715,2014,2014,AR,Arkansas,State Inpatient Data,Chronic Obstructive Pulmonary Disease,Hospitalization for chronic obstructive pulmon...,"cases per 10,000",Crude Rate,74.94,...,77.05,Gender,Female,"(34.748650124, -92.274490743)",5,COPD,COPD5_1,CRDRATE,GENDER,GENF
519716,2016,2016,CT,Connecticut,BRFSS,Cardiovascular Disease,Influenza vaccination among noninstitutionaliz...,%,Age-adjusted Prevalence,NaN,...,NaN,Race/Ethnicity,Hispanic,"(41.56266102, -72.649840952)",9,CVD,CVD9_2,AGEADJPREV,RACE,HIS


## Explore DataSources

In [17]:
Chronic_Diseases.get("DataSource").isna().sum()

0

### The dataset contains information from *32* different sources

In [18]:
# This dataset consists of 32 various sources
Chronic_Diseases.get("DataSource").value_counts().shape

(32,)

## Explore Stratification IDs

### Stratification ID1 contains 11 subcategories relating to `RACE`, `GENDER`, `OVERALL`

In [19]:
print(Chronic_Diseases.get("StratificationID1").isna().sum())
print(Chronic_Diseases.get("StratificationID1").value_counts())

0
OVR     77888
HIS     64034
BLK     64034
WHT     64034
GENF    60830
GENM    60830
OTH     46441
MRC     46441
AIAN    17593
API     16851
ASN       742
Name: StratificationID1, dtype: int64


## Next, let's see if there are redundant columns

In [20]:
Chronic_Diseases.columns

Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'DataSource',
       'Topic', 'Question', 'DataValueUnit', 'DataValueType', 'DataValue',
       'DataValueAlt', 'DatavalueFootnote', 'LowConfidenceLimit',
       'HighConfidenceLimit', 'StratificationCategory1', 'Stratification1',
       'GeoLocation', 'LocationID', 'TopicID', 'QuestionID', 'DataValueTypeID',
       'StratificationCategoryID1', 'StratificationID1'],
      dtype='object')

In [21]:
stratification_category = pd.DataFrame().assign(c1=Chronic_Diseases.get("Question"))
pd.set_option('max_colwidth', 400)
stratification_category

,c1
0,Amount of alcohol excise tax by beverage type (wine)
1,Amount of alcohol excise tax by beverage type (beer)
2,Amount of alcohol excise tax by beverage type (beer)
3,Amount of alcohol excise tax by beverage type (beer)
4,Amount of alcohol excise tax by beverage type (beer)
...,...
519713,Physical inactivity among adults aged >= 18 years with arthritis
519714,Meeting aerobic physical activity guidelines for additional and more extensive health benefits among adults aged >= 18 years
519715,Hospitalization for chronic obstructive pulmonary disease as first-listed diagnosis
519716,Influenza vaccination among noninstitutionalized adults aged >= 65 years with a history of coronary heart disease or stroke


In [22]:
alc_data = Chronic_Diseases[Chronic_Diseases['Topic'] == 'Alcohol']
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
alc_data

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,DatavalueFootnote,LowConfidenceLimit,HighConfidenceLimit,StratificationCategory1,Stratification1,GeoLocation,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1
0,2015,2015,US,United States,APIS,Alcohol,Amount of alcohol excise tax by beverage type (wine),$,US Dollars,NaN,NaN,No data available,NaN,NaN,Overall,Overall,NaN,59,ALC,ALC8_0_2,USD,OVERALL,OVR
1,2012,2012,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,0.09,0.09,NaN,NaN,NaN,Overall,Overall,"(38.907192, -77.036871)",11,ALC,ALC8_0_1,USD,OVERALL,OVR
2,2012,2012,GU,Guam,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,NaN,NaN,No data available,NaN,NaN,Overall,Overall,"(13.444304, 144.793731)",66,ALC,ALC8_0_1,USD,OVERALL,OVR
3,2012,2012,PR,Puerto Rico,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,NaN,NaN,No data available,NaN,NaN,Overall,Overall,"(18.220833, -66.590149)",72,ALC,ALC8_0_1,USD,OVERALL,OVR
4,2012,2012,US,United States,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,NaN,NaN,No data available,NaN,NaN,Overall,Overall,NaN,59,ALC,ALC8_0_1,USD,OVERALL,OVR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519576,2014,2014,OK,Oklahoma,BRFSS,Alcohol,Binge drinking frequency among adults aged >= 18 years who binge drink,Number,Age-adjusted Mean,6.2,6.20,NaN,3.1,9.2,Race/Ethnicity,"Multiracial, non-Hispanic","(35.472031356, -97.521070214)",40,ALC,ALC3_0,AGEADJMEAN,RACE,MRC
519646,2013,2013,NE,Nebraska,BRFSS,Alcohol,Binge drinking intensity among adults aged >= 18 years who binge drink,Number,Age-adjusted Mean,NaN,NaN,Sample size of denominator and/or age group for age-standardization is less than 50 or relative standard error is more than 30%,NaN,NaN,Race/Ethnicity,"Black, non-Hispanic","(41.641040988001, -99.365720623)",31,ALC,ALC4_0,AGEADJMEAN,RACE,BLK
519660,2014,2014,LA,Louisiana,NVSS,Alcohol,Chronic liver disease mortality,"cases per 100,000",Age-adjusted Rate,9.8,9.80,,8.9,10.7,Overall,Overall,"(31.312660644, -92.445680071)",22,ALC,ALC6_0,AGEADJRATE,OVERALL,OVR
519671,2013,2013,WA,Washington,BRFSS,Alcohol,Heavy drinking among women aged 18-44 years,%,Crude Prevalence,NaN,NaN,Sample size of denominator and/or age group for age-standardization is less than 50 or relative standard error is more than 30%,NaN,NaN,Race/Ethnicity,"Multiracial, non-Hispanic","(47.522278629, -120.47001079)",53,ALC,ALC5_2,CRDPREV,RACE,MRC


In [23]:
pd.set_option('display.max_rows', None)
alc_data['Question'].value_counts()

Chronic liver disease mortality                                           6135
Heavy drinking among adults aged >= 18 years                              5220
Binge drinking intensity among adults aged >= 18 years who binge drink    5220
Binge drinking prevalence among adults aged >= 18 years                   5220
Binge drinking frequency among adults aged >= 18 years who binge drink    5220
Binge drinking prevalence among women aged 18-44 years                    1950
Heavy drinking among women aged 18-44 years                               1950
Amount of alcohol excise tax by beverage type (distilled spirits)         110 
Binge drinking prevalence among youth                                     110 
Amount of alcohol excise tax by beverage type (beer)                      110 
Local authority to regulate alcohol outlet density                        110 
Amount of alcohol excise tax by beverage type (wine)                      110 
Alcohol use among youth                             

In [24]:
pd.set_option('display.max_rows', 10)
alc_mod_data = alc_data[alc_data['DatavalueFootnote'] != 'No data available']
alc_mod_data = alc_mod_data[alc_mod_data['DatavalueFootnote'] != 'State directly controlled the sale of (beer/wine/distilled spirits) at the retail and/or wholesale levels. State prices for (beer/wine/distilled spirits) combined both markups and taxes, so tax rates for this beverage type could not be determined.']
alc_mod_data = alc_mod_data[alc_mod_data['DatavalueFootnote'] != 'Sample size of denominator and/or age group for age-standardization is less than 50 or relative standard error is more than 30%']
alc_mod_data = alc_mod_data[alc_mod_data['DatavalueFootnote'] != 'Data not shown because of too few respondents or cases']
alc_mod_data

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,DatavalueFootnote,LowConfidenceLimit,HighConfidenceLimit,StratificationCategory1,Stratification1,GeoLocation,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1
1,2012,2012,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,0.09,0.09,NaN,NaN,NaN,Overall,Overall,"(38.907192, -77.036871)",11,ALC,ALC8_0_1,USD,OVERALL,OVR
14,2015,2015,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,0.09,0.09,NaN,NaN,NaN,Overall,Overall,"(38.907192, -77.036871)",11,ALC,ALC8_0_1,USD,OVERALL,OVR
17,2012,2012,OR,Oregon,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,0.08,0.08,NaN,NaN,NaN,Overall,Overall,"(44.567449424, -120.155031326)",41,ALC,ALC8_0_1,USD,OVERALL,OVR
18,2012,2012,MT,Montana,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,0.14,0.14,NaN,NaN,NaN,Overall,Overall,"(47.066528972, -109.424420645)",30,ALC,ALC8_0_1,USD,OVERALL,OVR
21,2015,2015,NE,Nebraska,APIS,Alcohol,Amount of alcohol excise tax by beverage type (wine),$,US Dollars,0.95,0.95,NaN,NaN,NaN,Overall,Overall,"(41.641040988001, -99.365720623)",31,ALC,ALC8_0_2,USD,OVERALL,OVR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519515,2015,2015,WA,Washington,BRFSS,Alcohol,Heavy drinking among adults aged >= 18 years,%,Age-adjusted Prevalence,3.9,3.90,NaN,2.5,5.9,Race/Ethnicity,"Other, non-Hispanic","(47.522278629, -120.47001079)",53,ALC,ALC5_1,AGEADJPREV,RACE,OTH
519537,2015,2015,SD,South Dakota,BRFSS,Alcohol,Binge drinking frequency among adults aged >= 18 years who binge drink,Number,Age-adjusted Mean,3.7,3.70,NaN,3.0,4.3,Race/Ethnicity,"Other, non-Hispanic","(44.353130053, -100.373530637)",46,ALC,ALC3_0,AGEADJMEAN,RACE,OTH
519576,2014,2014,OK,Oklahoma,BRFSS,Alcohol,Binge drinking frequency among adults aged >= 18 years who binge drink,Number,Age-adjusted Mean,6.2,6.20,NaN,3.1,9.2,Race/Ethnicity,"Multiracial, non-Hispanic","(35.472031356, -97.521070214)",40,ALC,ALC3_0,AGEADJMEAN,RACE,MRC
519660,2014,2014,LA,Louisiana,NVSS,Alcohol,Chronic liver disease mortality,"cases per 100,000",Age-adjusted Rate,9.8,9.80,,8.9,10.7,Overall,Overall,"(31.312660644, -92.445680071)",22,ALC,ALC6_0,AGEADJRATE,OVERALL,OVR


In [25]:
alc_tax_data = alc_mod_data[(alc_mod_data['Question'] == 'Amount of alcohol excise tax by beverage type (beer)') |
                           (alc_mod_data['Question'] == 'Amount of alcohol excise tax by beverage type (wine)') |
                           (alc_mod_data['Question'] == 'Amount of alcohol excise tax by beverage type (distilled spirits)')]
alc_tax_data

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,DatavalueFootnote,LowConfidenceLimit,HighConfidenceLimit,StratificationCategory1,Stratification1,GeoLocation,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1
1,2012,2012,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,0.09,0.09,NaN,NaN,NaN,Overall,Overall,"(38.907192, -77.036871)",11,ALC,ALC8_0_1,USD,OVERALL,OVR
14,2015,2015,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,0.09,0.09,NaN,NaN,NaN,Overall,Overall,"(38.907192, -77.036871)",11,ALC,ALC8_0_1,USD,OVERALL,OVR
17,2012,2012,OR,Oregon,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,0.08,0.08,NaN,NaN,NaN,Overall,Overall,"(44.567449424, -120.155031326)",41,ALC,ALC8_0_1,USD,OVERALL,OVR
18,2012,2012,MT,Montana,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,0.14,0.14,NaN,NaN,NaN,Overall,Overall,"(47.066528972, -109.424420645)",30,ALC,ALC8_0_1,USD,OVERALL,OVR
21,2015,2015,NE,Nebraska,APIS,Alcohol,Amount of alcohol excise tax by beverage type (wine),$,US Dollars,0.95,0.95,NaN,NaN,NaN,Overall,Overall,"(41.641040988001, -99.365720623)",31,ALC,ALC8_0_2,USD,OVERALL,OVR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435143,2012,2012,CA,California,APIS,Alcohol,Amount of alcohol excise tax by beverage type (distilled spirits),$,US Dollars,3.3,3.30,NaN,NaN,NaN,Overall,Overall,"(37.638640123, -120.999999538)",6,ALC,ALC8_0_3,USD,OVERALL,OVR
446483,2012,2012,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type (distilled spirits),$,US Dollars,1.5,1.50,NaN,NaN,NaN,Overall,Overall,"(38.907192, -77.036871)",11,ALC,ALC8_0_3,USD,OVERALL,OVR
451057,2012,2012,TX,Texas,APIS,Alcohol,Amount of alcohol excise tax by beverage type (distilled spirits),$,US Dollars,2.4,2.40,NaN,NaN,NaN,Overall,Overall,"(31.827240407, -99.426770206)",48,ALC,ALC8_0_3,USD,OVERALL,OVR
492858,2012,2012,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type (wine),$,US Dollars,0.3,0.30,NaN,NaN,NaN,Overall,Overall,"(38.907192, -77.036871)",11,ALC,ALC8_0_2,USD,OVERALL,OVR


In [26]:
alc_tax_data = alc_tax_data.drop(columns = ['DatavalueFootnote', 'LowConfidenceLimit', 'HighConfidenceLimit',
                                           'StratificationCategory1', 'Stratification1', 'GeoLocation',
                                           'LocationID', 'StratificationCategoryID1', 'StratificationID1',
                                           'YearStart', 'YearEnd', 'QuestionID', 'DataValueTypeID'])
alc_tax_data

,LocationAbbr,LocationDesc,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,TopicID
1,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,0.09,0.09,ALC
14,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,0.09,0.09,ALC
17,OR,Oregon,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,0.08,0.08,ALC
18,MT,Montana,APIS,Alcohol,Amount of alcohol excise tax by beverage type (beer),$,US Dollars,0.14,0.14,ALC
21,NE,Nebraska,APIS,Alcohol,Amount of alcohol excise tax by beverage type (wine),$,US Dollars,0.95,0.95,ALC
...,...,...,...,...,...,...,...,...,...,...
435143,CA,California,APIS,Alcohol,Amount of alcohol excise tax by beverage type (distilled spirits),$,US Dollars,3.3,3.30,ALC
446483,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type (distilled spirits),$,US Dollars,1.5,1.50,ALC
451057,TX,Texas,APIS,Alcohol,Amount of alcohol excise tax by beverage type (distilled spirits),$,US Dollars,2.4,2.40,ALC
492858,DC,District of Columbia,APIS,Alcohol,Amount of alcohol excise tax by beverage type (wine),$,US Dollars,0.3,0.30,ALC
